In [1]:
import pandas as pd
import numpy as np


In [2]:
jester = pd.read_csv("data/jester-data-1.csv", header=None).drop(0, axis=1).T.reset_index(drop=True).T
jester = jester.loc[range(200)]
jester

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5.73,0.00,-0.58,1.75,2.86,7.23,2.04,-4.27,3.11,7.14,...,1.70,4.66,1.65,5.58,5.73,3.50,0.00,-2.04,2.91,1.84
196,1.80,5.39,8.06,-0.10,4.81,4.66,-7.09,-7.72,3.69,4.47,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
197,2.62,-3.79,-1.17,-0.44,-1.02,5.05,-3.69,2.18,3.30,-0.73,...,1.41,-2.52,-0.53,-1.89,1.17,-2.04,-1.17,-0.73,0.73,0.44
198,99.00,99.00,99.00,99.00,7.91,99.00,3.64,-1.89,99.00,99.00,...,2.96,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [42]:
def pearson_corr(x1, x2):
    x1_cent = x1 - x1.mean()
    x2_cent = x2 - x2.mean()

    num = (x1_cent * x2_cent).sum()
    denom = np.sqrt((x1_cent**2).sum()) * np.sqrt((x2_cent**2).sum())

    return num / denom

In [41]:
def similarity_matrix(data, method = "user"):
    sim = pd.DataFrame()
    if method == "user":
        users = list(data.index)
        sim = pd.DataFrame(index=data.index, columns=data.index)
        for i in users:
            for j in users:
                if i == j:
                    sim.loc[i, j] = np.nan
                else:
                    sim.loc[i, j] = pearson_corr(data.loc[i, :], data.loc[j, :])
    elif method == "item":
        items = list(data.columns)
        sim = pd.DataFrame(index=data.columns, columns=data.columns)
        for i in items:
            for j in items:
                if i == j:
                    sim.loc[i, j] = np.nan
                else:
                    sim.loc[i, j] = pearson_corr(data.loc[:, i], data.loc[:, j])

    return sim

In [94]:
def mean_utility(orig, sim, user, item, method, n = None):
    if method == "user":
        users_of_item = orig.loc[:, item].copy()
        users_of_item.loc[user] = np.nan
        if n is not None:
            users_sim = sim.loc[user][~users_of_item.isna()].copy()
            neighbors = pd.Series(users_sim.index)[np.argpartition(-users_sim, n)[:n]]
            users_of_item = users_of_item.loc[neighbors]
        return max(min(users_of_item.mean(), 10), -10)
    elif method == "item":
        items_of_user = orig.loc[user, :].copy()
        items_of_user.loc[item] = np.nan
        if n is not None:
            items_sim = sim.loc[item][~items_of_user.isna()].copy()
            neighbors = pd.Series(items_sim.index)[np.argpartition(-items_sim, n)[:n]]
            items_of_user = items_of_user.loc[neighbors]
        return max(min(items_of_user.mean(), 10), -10)
    return np.nan

In [100]:
def wtd_sum(orig, sim, user, item, method, n = None):
    if method == "user":
        users_of_item = orig.loc[:, item].copy()
        users_of_item.loc[user] = np.nan
        users_sim = sim.loc[user].copy()
        if n is not None:
            users_sim = users_sim[~users_of_item.isna()].copy()
            neighbors = pd.Series(users_sim.index)[np.argpartition(-users_sim, n)[:n]]
            users_of_item = users_of_item.loc[neighbors]
        return max(min((users_of_item * users_sim).sum() / users_sim.sum(), 10), -10)
    elif method == "item":
        items_of_user = orig.loc[user, :].copy()
        items_of_user.loc[item] = np.nan
        items_sim = sim.loc[item].copy()
        if n is not None:
            items_sim = items_sim[~items_of_user.isna()].copy()
            neighbors = pd.Series(items_sim.index)[np.argpartition(-items_sim, n)[:n]]
            items_of_user = items_of_user.loc[neighbors]
        return max(min((items_of_user * items_sim).sum() / items_sim.sum(), 10), -10)
    return np.nan

In [ ]:
def adj_wtd_sum(orig, sim, user, item, method, n = None):
    if method == "user":
        users_of_item = orig.loc[:, item].copy()
        users_of_item.loc[user] = np.nan
        user_means = orig.mean(axis=1)
        user_mean = user_means.loc[user]
        users_sim = sim.loc[user].copy()
        if n is not None:
            users_sim = users_sim[~users_of_item.isna()].copy()
            neighbors = pd.Series(users_sim.index)[np.argpartition(-users_sim, n)[:n]]
            users_of_item = users_of_item.loc[neighbors]
        return max(min(user_mean + (((users_of_item - user_means) * users_sim).sum() / users_sim.sum()), 10), -10)
    elif method == "item":
        items_of_user = orig.loc[user, :].copy()
        items_of_user.loc[item] = np.nan
        item_means = orig.mean(axis=0)
        item_mean = item_means.loc[item]
        items_sim = sim.loc[user].copy()
        if n is not None:
            items_sim = items_sim[~items_of_user.isna()].copy()
            neighbors = pd.Series(items_sim.index)[np.argpartition(-items_sim, n)[:n]]
            items_of_user = items_of_user.loc[neighbors]
        return max(min(item_mean + (((items_of_user - item_means) * items_sim).sum() / items_sim.sum()), 10), -10)

In [101]:
def recommend_all(data, method = "user", func = mean_utility, n = None):
    results = pd.DataFrame(index=data.index, columns=data.columns)
    orig = data.replace(99, np.nan)

    sim = similarity_matrix(data, method=method)
    for user in orig.index:
        for item in orig.columns:
            results.loc[user, item] = func(orig, sim, user, item, method, n=n)

    return results

In [102]:
recommend_all(jester, func=wtd_sum, n=5)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.006678,-0.010389,-0.093944,0.243209,0.016613,0.161922,0.000098,-0.013714,0.056361,-0.236284,...,0.161936,0.409236,0.830469,1.24435,1.864275,1.056014,1.415845,-0.047943,-0.335703,1.628602
1,0.190694,0.249508,0.061972,1.024439,0.416513,0.939637,-0.014153,0.042232,0.136766,-0.366439,...,-2.545018,10,10,-10,-4.42067,1.30274,4.906394,-2.224764,1.003864,-1.745774
2,0.406368,0.401169,0.240275,0.542398,0.281827,0.625486,0.102593,0.284987,-0.184311,0.322004,...,-9.611916,-5.905053,6.448051,0.007991,10,5.697162,1.819278,1.893814,0.200144,-8.031775
3,0.08031,0.051279,0.259759,0.762673,0.084124,0.817751,0.220433,0.076243,-0.100329,0.336877,...,-6.60532,-1.156017,5.203639,1.823647,10,6.411932,2.068917,1.428585,2.709418,-7.084662
4,-0.20893,-0.037285,0.09287,-0.294286,0.414427,-0.080591,-0.000101,0.077193,0.041789,0.119064,...,0.78061,0.078597,-0.281222,-0.919627,-0.641723,-0.478006,-0.174652,-0.269116,-0.847251,0.90709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-0.46795,-0.134398,0.969486,-5.827978,-0.034197,-0.092837,-0.052519,0.356763,2.192665,0.241163,...,-0.181517,-0.428889,1.271524,0.969633,0.048128,-0.245643,0.715363,0.206414,-0.020721,0.185708
196,0.1278,0.069889,0.084256,0.202552,0.025968,0.255473,-0.062702,0.006001,0.388227,-0.060941,...,0.342412,0.431968,0.872705,1.274153,1.709735,1.097136,1.366896,-0.066075,-0.755345,1.719906
197,-0.529302,0.37536,-0.144308,1.090704,-0.122935,-0.036538,-0.107195,0.028809,-0.426459,-0.256251,...,1.02466,-0.226577,1.429112,0.388685,0.795558,0.838549,-0.951778,-0.24233,-1.599678,0.970323
198,-0.622836,-0.063277,0.21476,0.769048,0.224044,0.641824,0.073138,-0.079202,-0.185802,0.391793,...,-10,-0.942199,4.794136,1.488775,9.524631,4.231917,1.716032,2.720303,0.589141,-9.466833


In [104]:
jester_stack = jester.stack().reset_index()

,level_0,level_1,0
0,0,0,-7.82
1,0,1,8.79
2,0,2,-9.66
3,0,3,-8.16
4,0,4,-7.52
...,...,...,...
19995,199,95,99.00
19996,199,96,99.00
19997,199,97,99.00
19998,199,98,99.00
